## Prepare and read data

In [1]:
import gc
import itertools
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [4]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Feature Engineering

In [5]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [6]:
original_cols = train_df.columns.tolist()

In [7]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [8]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [9]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

In [10]:
non_basic_nume_cols = []
non_basic_cat_cols = []

In [11]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### rank feature

In [12]:
def add_rank_feature(df):
    rank_prefix_list = [
        "AssessBenefits",
        "AssessJob",
        "JobContactPriorities",
        "JobEmailPriorities",
        "AdsPriorities",
    ]
    for prefix in tqdm(rank_prefix_list):
        rank_cols = [c for c in df.columns if prefix in c]
        col_pairs = itertools.combinations(rank_cols, 2)
        for col_a, col_b in col_pairs:
            df[f"rank_diff_{prefix}_{col_a}_{col_b}"] = (df[col_a] - df[col_b]) / np.log2(df[[col_a, col_b]].max(axis=1))
    return df

In [13]:
train_df = add_rank_feature(train_df)
test_df = add_rank_feature(test_df)

### Multi-category encoding

In [14]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)
    # ohe_featureはcategoryとnumerical両方として扱う
    nume_cols += feat_cols
    cat_cols += feat_cols
    # non_basic_nume_cols.append(f"sum_answer_{c}")
    # SVD
    svd = TruncatedSVD(n_components=2, random_state=2020)
    all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
    train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)

### Simple-category encoding

In [15]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

In [16]:
train_null_pred = pd.read_csv("../data/train_null_pred.csv")
test_null_pred = pd.read_csv("../data/test_null_pred.csv")

In [17]:
for c in train_null_pred.columns:
    train_df[c] = train_null_pred[c]
    test_df[c] = test_null_pred[c]

### Reduce data memory

In [18]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [19]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [20]:
train_df.shape, test_df.shape

((33857, 659), (11259, 658))

### Change Category Name

In [21]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [22]:
def get_basic_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [23]:
def make_agg_feature(train_df, test_df):
    imp_feats = get_basic_importance_cols(use_num=50)
    imp_cat_cols = [c for c in cat_cols if c in imp_feats] + non_basic_cat_cols
    imp_nume_cols = [c for c in nume_cols if c in imp_feats] + non_basic_nume_cols
    print(f"use cat col: {len(imp_cat_cols)}  nume col: {len(imp_nume_cols)}")
    #imp_nume_cols += [c for c in train_df.columns if c[:8] == "sum_answ"]
    all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    for cat_col in tqdm(imp_cat_cols):
        for nume_col in imp_nume_cols:
            # one-hotは同じカテゴリの場合がある
            if cat_col == nume_col:
                continue
            all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
                all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
            all_df[f"agg_std_{cat_col}_{nume_col}"] = \
                all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
            all_df[f"agg_max_{cat_col}_{nume_col}"] = \
                all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
            all_df[f"agg_min_{cat_col}_{nume_col}"] = \
                all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
            all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
                all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
            all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
                all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
    train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
    test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
    del all_df
    gc.collect()
    return train_df, test_df

In [24]:
train_df, test_df = make_agg_feature(train_df, test_df)

use cat col: 27  nume col: 10


### Make fold index(for target encoding)

In [25]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [26]:
@feature_cache(reset=False)
def target_encoding_after_imputation(train_df, test_df):
    te_cols = [c for c in train_df.columns if c in cat_cols]
    for c in tqdm(te_cols):
        new_col = "te_" + c
        train_df[new_col] = 0
        test_df[new_col] = 0
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
    return train_df, test_df

In [27]:
train_df, test_df = target_encoding_after_imputation(train_df, test_df)

In [28]:
@feature_cache(reset=False)
def multiple_target_encoding_after_imputation(train_df, test_df):
     # multiple target encoding
    multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
    imp_feats = get_basic_importance_cols(use_num=30)
    multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
    combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

    for col_a, col_b in tqdm(combi_multi_te_cols):
        new_col = "te_" + col_a + "__" + col_b
        train_df[new_col] = 0
        test_df[new_col] = 0
        train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
        test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
        del train_df["tmp"], test_df["tmp"]
        gc.collect()
    return train_df, test_df

In [29]:
train_df, test_df = multiple_target_encoding_after_imputation(train_df, test_df)

In [30]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

2659


## Training

In [31]:
lgb_params = {
            'objective': 'poisson',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [32]:
def feature_selection(train_df, use_cols, n_features=1000):
    df = train_df.sample(30000, random_state=2020)
    train_dataset = lgb.Dataset(
        df.loc[:, use_cols],
        label = df.loc[:, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                2000,
                valid_sets = [train_dataset],
                verbose_eval=200,
                early_stopping_rounds = None,
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    select_features = imp_df.sort_values(["gain"], ascending=False).iloc[: n_features]["feature"].tolist()
    return select_features

In [33]:
def null_preds(train_df, test_df, use_cols):
    target_cols = [
        "SalaryType",
        "YearsCodingProf",
        "Age",
    ]
    all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    _lgb_params = lgb_params.copy()
    _lgb_params["objective"] = "multiclass"
    _lgb_params["metric"] = "multi_logloss" 
    for target_col in target_cols:
        print("-"*30)
        print(target_col)
        print("-"*30)

        _use_cols = [c for c in use_cols if c != target_col]
        trnval_idx = all_df[all_df[target_col] >= 0].index
        null_idx = all_df[all_df[target_col] == -1].index
        _train_df = all_df.loc[trnval_idx].reset_index(drop=True)
        _train_X, _valid_X, _train_y, _valid_y = train_test_split(
            _train_df[_use_cols], _train_df[target_col], test_size=0.2, shuffle=True, random_state=2020)
        _lgb_params["num_classes"] = _train_y.nunique()
        print(_train_y.nunique())
        train_dataset = lgb.Dataset(
            _train_X,
            label = _train_y
        )
        valid_dataset = lgb.Dataset(
            _valid_X,
            label = _valid_y
        )
        model = lgb.train(
                    _lgb_params,
                    train_dataset,
                    10000,
                    valid_sets = [train_dataset, valid_dataset],
                    verbose_eval=100,
                    early_stopping_rounds = 300,
                    #feval = eval_f1,
                    #callbacks = [log_callback],
        )
        all_df.loc[null_idx, target_col] = model.predict(all_df.loc[null_idx, _use_cols]).argmax(axis=1)
    train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
    test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
    return train_df, test_df

In [34]:
# train_df, test_df = null_preds(train_df, test_df, use_cols)

In [35]:
"""
null_pred_cols = [
    "SalaryType",
    "YearsCodingProf",
    "Age",
]
train_df[null_pred_cols].to_csv("../data/train_null_pred.csv", index=False)
test_df[null_pred_cols].to_csv("../data/test_null_pred.csv", index=False)
"""

'\nnull_pred_cols = [\n    "SalaryType",\n    "YearsCodingProf",\n    "Age",\n]\ntrain_df[null_pred_cols].to_csv("../data/train_null_pred.csv", index=False)\ntest_df[null_pred_cols].to_csv("../data/test_null_pred.csv", index=False)\n'

In [36]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

# use_cols = feature_selection(train_df, use_cols, n_features=1000)


for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                10000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 300,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 22735.7	valid_1's rmse: 23067.4
[200]	training's rmse: 20869.2	valid_1's rmse: 21640.3
[300]	training's rmse: 20102.1	valid_1's rmse: 21204.6
[400]	training's rmse: 19612	valid_1's rmse: 20988.6
[500]	training's rmse: 19263.3	valid_1's rmse: 20844.5
[600]	training's rmse: 18949.5	valid_1's rmse: 20749.2
[700]	training's rmse: 18684.5	valid_1's rmse: 20673.8
[800]	training's rmse: 18425.8	valid_1's rmse: 20614.3
[900]	training's rmse: 18203.3	valid_1's rmse: 20557.7
[1000]	training's rmse: 17999.3	valid_1's rmse: 20517.7
[1100]	training's rmse: 17792.9	valid_1's rmse: 20491.6
[1200]	training's rmse: 17597.2	valid_1's rmse: 20461.1
[1300]	training's rmse: 17410.2	valid_1's rmse: 20441.9
[1400]	training's rmse: 17243.1	valid_1's rmse: 20429.9
[1500]	training's rmse: 17070	valid_1's rmse: 20408.8
[1600]	training's rmse: 16912.5	valid_1's rmse: 20396.3
[1700]	training's rmse: 16752	valid_1's rmse: 20

In [44]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20163.675773404197

In [38]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__Age,5.416772e+08
1,te_Country__ohe_DevType_Student,4.058566e+08
2,te_Country__YearsCodingProf,3.392185e+08
3,te_YearsCodingProf__CurrencySymbol,2.667667e+08
4,te_Country__ohe_CommunicationTools_Confluence,2.275610e+08
5,te_Country__CompanySize,1.918541e+08
6,te_Country__YearsCoding,1.815849e+08
7,te_Country__FormalEducation,1.725222e+08
8,te_Country__Dependents,1.588446e+08
9,te_Country__Employment,1.586645e+08


In [39]:
len(use_cols)

2659

## Prediction

In [40]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [41]:
test_pred

array([ 68333.80367045,  92730.29528771,  84960.24614034, ...,
        88306.23995344,  82873.72413037, 120223.86727425])

In [42]:
sub_df = pd.read_csv("../input/submit.csv")

In [43]:
sub_df["Salary"] = test_pred

In [45]:
sub_df.to_csv("../predict/032_null_pred_20163.csv", index=False)